## Data Downloader for ERA5 and GLoFAS data

defines some methods to download era5 data

In [1]:
import cdsapi

In [18]:
def era5_retrieval(savename, base_level='pressure', form='netcdf', area_str='danube',
                   variable='geopotential', pressure_level='700', time_start='2017:05',
                   time_end='2017:07'):
    # download era5 data with the cdsapi
    # define some vars
    area_dict = {'danube': '50/7/47/20',
                 'asia': '55/-140/0/35',
                 'usa': '50/-125/25/-70'
                }
    area = area_dict[area_str]
    
    # build iterator based on time range
    # data request efficiency is highest when executed on a monthly basis
    years = range(int(time_start.split(':')[0]), int(time_end.split(':')[0])+1)
    months = range(int(time_start.split(':')[1]), int(time_end.split(':')[1])+1)

    # loop over time range
    for y in years:
        for m in months:
            # start a request for one month
#            request_data(area=area, variable=variable, pressure_level=pressure_level, year=y, month=m)    
    

    return None

def request_data(savename='era5_example', base_level='pressure', form='netcdf', area='50/7/47/20',
                 variable='geopotential', pressure_level='700', year='2000', month='12'):
    c = cdsapi.Client()
    if base_level == 'pressure':
        c.retrieve(f"reanalysis-era5-{base_level}-levels", {
                "product_type":   "reanalysis",
                "format":         form,
                "area":           area, # N/W/S/E
                "variable":       variable,
                "pressure_level": pressure_level,
                "year":           year,
                "month":          month,
                "day":            ["01", "02", "03", "04",
                                   "05", "06", "07", "08",
                                   "09", "10", "11", "12",
                                   "13", "14", "15", "16",
                                   "17", "18", "19", "20",
                                   "21", "22", "23", "24",
                                   "25", "26", "27", "28",
                                   "29", "30", "31"],
                "time":           ["00", "01", "02", "03", "04", "05",
                                  "06", "07", "08", "09", "10", "11",
                                  "12", "13","14", "15", "16", "17",
                                  "18", "19", "20","21", "22", "23"]
            }, f'{savename}.nc')
    elif base_level == 'single':
        c.retrieve(f"reanalysis-era5-{base_level}-levels", {
                "product_type":   "reanalysis",
                "format":         form,
                "area":           area, # N/W/S/E
                "variable":       variable,
                "year":           year,
                "month":          month,
                "day":            ["01", "02", "03", "04",
                                   "05", "06", "07", "08",
                                   "09", "10", "11", "12",
                                   "13", "14", "15", "16",
                                   "17", "18", "19", "20",
                                   "21", "22", "23", "24",
                                   "25", "26", "27", "28",
                                   "29", "30", "31"],
                "time":           ["00", "01", "02", "03", "04", "05",
                                  "06", "07", "08", "09", "10", "11",
                                  "12", "13","14", "15", "16", "17",
                                  "18", "19", "20","21", "22", "23"]
            }, f'{savename}.nc')
            
    return None

In [19]:



era5_retrieval('test', area_str='danube')


2017 5
2017 6
2017 7


### download request for the data of interest on pressure levels

In [ ]:
# choose area: 'danube', 'asia', 'usa'
area = 'danube'

# variables
variable = ['geopotential', 'temperature', 'specific humidity']

# pressure levels
base_level = 'pressure' # 'pressure' or 'single'
pressure_level = ['850', '700', '500']


# define time range: start end in the format YYYY:MM
time_start = '2017:05' # full range start: '1981:01'
time_end = '2017:07' # full range end: '2017:12'




